In [1]:
print(_IMPORTS)

import datetime as dt
from collections import defaultdict, Counter, namedtuple
from itertools import product, combinations, permutations
import json
import os
import pickle
import warnings

from IPython.display import HTML, display, set_matplotlib_formats
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sqlalchemy import create_engine
from tqdm.auto import tqdm

# INITIALIZING
# warnings
warnings.filterwarnings('ignore')
# pictures
sns.set()
plt.style.use('seaborn-bright')
plt.rcParams['figure.figsize'] = 10, 6
# pandas
pd.set_option('display.max_columns', 150)
pd.set_option('display.max_rows', 100)
tqdm.pandas()

# Autoreload
get_ipython().magic(u"%reload_ext autoreload")
get_ipython().magic(u"%autoreload 2")
get_ipython().magic(u"%load_ext line_profiler")



In [2]:
import re

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import *

from sklearn.model_selection import train_test_split

### Sampling on patients

In [4]:
PATH = "../data/raw/"
PATH_PROCESSED = "../data/processed/"
PATH_DATASETS = "../data/train/"

In [5]:
SAMPLE_SIZE = 10000
RANDOM_SEED = 1

In [6]:
patients = pd.read_csv(os.path.join(PATH, "PATIENTS.csv.gz"))
patients.EXPIRE_FLAG.value_counts()

0    30761
1    15759
Name: EXPIRE_FLAG, dtype: int64

In [7]:
#ensuring every patient is unique
print(f"{patients.SUBJECT_ID.nunique()} unique patients in {len(patients)} rows")
#sampling random patients
patients_sample = patients.sample(n=SAMPLE_SIZE, random_state=RANDOM_SEED)

46520 unique patients in 46520 rows


In [8]:
SAMPLE_IDS = set(patients_sample.SUBJECT_ID)

with open(os.path.join(PATH_PROCESSED, "SAMPLE_IDS.json"), 'w') as f:
    json.dump({'ids': list(SAMPLE_IDS)}, f)
    
with open(os.path.join(PATH_PROCESSED, "SAMPLE_IDS.json"), 'r') as f:
    SAMPLE_IDS = set(json.load(f)['ids'])

In [9]:
patients_sample = patients[patients.SUBJECT_ID.isin(SAMPLE_IDS)]

In [10]:
patients_sample.head()

,ROW_ID,SUBJECT_ID,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG
3,237,252,M,2078-03-06 00:00:00,NaN,NaN,NaN,0
10,244,261,M,2025-08-04 00:00:00,2102-06-29 00:00:00,2102-06-29 00:00:00,2102-06-29 00:00:00,1
23,629,666,F,2069-08-26 00:00:00,2139-11-20 00:00:00,NaN,2139-11-20 00:00:00,1
25,631,668,F,2096-08-18 00:00:00,2183-07-10 00:00:00,2183-07-10 00:00:00,2183-07-10 00:00:00,1
34,640,677,M,2195-08-12 00:00:00,NaN,NaN,NaN,0


In [11]:
DECEASED_TO_DATE = patients_sample[patients_sample.EXPIRE_FLAG == 1]\
    .set_index('SUBJECT_ID').DOD.map(lambda x: pd.to_datetime(x).date()).to_dict()

In [12]:
def get_data_for_sample(sample_ids: set, 
                        file_name: str, 
                        chunksize: int = 10_000) -> pd.DataFrame:
    """Get the data only relevant for the sample."""
    full_path = os.path.join(PATH, file_name)
    iterator = pd.read_csv(full_path, iterator=True, chunksize=chunksize)
    return pd.concat([chunk[chunk.SUBJECT_ID.isin(SAMPLE_IDS)] for chunk in tqdm(iterator)])

In [13]:
### Incomplete: Using AWS glue, for now I will do stuff locally
# import boto3
# Creating the low level functional client
# client = boto3.client(
#     'glue',
#     aws_access_key_id = '',
#     aws_secret_access_key = '',
#     region_name = 'us-east-1'
# )
# clientResponse = client.get_table(DatabaseName="mimiciii",Name="admissions")

### Build features

All features in feature_prepocessed form are features with columns ['SUBJECT_ID', 'FEATURE_NAME', 'DATE', 'VALUE], which can be later used for any of the aggregations we'd like.

In [14]:
admissions = get_data_for_sample(SAMPLE_IDS, "ADMISSIONS.csv.gz")

0it [00:00, ?it/s]

In [15]:
admissions['ADMITTIME'] = pd.to_datetime(admissions.ADMITTIME).dt.date

In [16]:
diagnoses = get_data_for_sample(SAMPLE_IDS, "DIAGNOSES_ICD.csv.gz")

0it [00:00, ?it/s]

In [17]:
lab_results = get_data_for_sample(SAMPLE_IDS, "LABEVENTS.csv.gz", chunksize=100_000)

0it [00:00, ?it/s]

In [18]:
meds = get_data_for_sample(SAMPLE_IDS, "PRESCRIPTIONS.csv.gz")

0it [00:00, ?it/s]

#### Diagnoses

In [19]:
diagnoses['ICD9_CODE'] = "ICD9_" + diagnoses['ICD9_CODE']

In [20]:
adm_cols = ['SUBJECT_ID', 'HADM_ID', 'ADMITTIME']
diagnoses = diagnoses.merge(admissions[adm_cols], on=['SUBJECT_ID', 'HADM_ID'])

In [21]:
dropper = ['ROW_ID', 'SEQ_NUM', 'HADM_ID']
renamer = {'ICD9_CODE': 'FEATURE_NAME', 'ADMITTIME': 'DATE'}
diag_preprocessed = diagnoses.drop(columns=dropper).rename(columns=renamer)
diag_preprocessed['VALUE'] = 1

In [22]:
diag_preprocessed.head()

,SUBJECT_ID,FEATURE_NAME,DATE,VALUE
0,109,ICD9_40301,2141-09-18,1
1,109,ICD9_486,2141-09-18,1
2,109,ICD9_58281,2141-09-18,1
3,109,ICD9_5855,2141-09-18,1
4,109,ICD9_4254,2141-09-18,1


#### Labs

In [23]:
lab_results['DATE'] = pd.to_datetime(lab_results['CHARTTIME']).dt.date

In [24]:
lab_results['FEATURE_NAME'] = "LAB_" + lab_results['ITEMID'].astype(str)

In [25]:
dropper = ['ROW_ID', 'HADM_ID', 'VALUE', 'VALUEUOM', 'FLAG', 'ITEMID', 'CHARTTIME']
renamer = {'VALUENUM': 'VALUE'}
lab_preprocessed = lab_results.drop(columns=dropper).rename(columns=renamer)

In [26]:
lab_preprocessed.head()

,SUBJECT_ID,VALUE,DATE,FEATURE_NAME
4912,11,13.0,2178-04-15,LAB_50868
4913,11,25.0,2178-04-15,LAB_50882
4914,11,104.0,2178-04-15,LAB_50902
4915,11,0.7,2178-04-15,LAB_50912
4916,11,NaN,2178-04-15,LAB_50920


#### Meds

In [27]:
meds = meds[meds.ENDDATE.notna()]

In [28]:
meds['DATE'] = pd.to_datetime(meds['ENDDATE']).dt.date

In [29]:
def find_mean_dose(dose: str) -> float:
    if pd.isnull(dose):
        return 0
    try:
        cleaned = re.sub(r'[A-Za-z,>< ]', '', dose)
        parts = cleaned.split('-')
        return np.array(parts).astype(float).mean()
    except:
        print(dose)

In [30]:
meds['VALUE'] = meds['DOSE_VAL_RX'].map(find_mean_dose)

-0.5-2
-15-30
1.25/3
1%
1.26mg/6
1.26mg/6


In [31]:
meds['FEATURE_NAME'] = "MED_" + meds['GSN'].astype(str)

In [32]:
dropper = [col for col in meds.columns if col not in {'SUBJECT_ID', 'DATE', 'FEATURE_NAME', 'VALUE'}]
meds_preprocessed = meds.drop(columns=dropper).rename(columns=renamer)

In [33]:
meds_preprocessed.head()

,SUBJECT_ID,DATE,VALUE,FEATURE_NAME
596,11,2178-04-17,0.5,MED_046288
597,11,2178-04-17,10.0,MED_006778
598,11,2178-04-17,0.5,MED_nan
599,11,2178-04-21,4.0,MED_006789
600,11,2178-04-21,4.0,MED_061716


#### Notes

Here we can preprocess notes. Later the same things can be done using Spark.

In [34]:
def get_notes(sample_ids: set = None,
              note_path: str = "NOTEEVENTS.csv.gz",
              chunksize: int = 10_000) -> pd.DataFrame:
    """Get all notes or only those relevant for the sample."""
    if sample_ids is None:
        return pd.read_csv(os.path.join(PATH, note_path))
    return get_data_for_sample(sample_ids, note_path, chunksize)

In [35]:
notes_sample = get_notes(SAMPLE_IDS)
# notes_sample.to_csv(os.path.join(PATH_PROCESSED, 'SAMPLE_NOTES.csv'), index=False)

notes = pd.read_csv(os.path.join(PATH_PROCESSED, 'SAMPLE_NOTES.csv'))

0it [00:00, ?it/s]

In [36]:
notes.head()

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
0,223,5350,169684.0,2143-04-30,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2143-4-25**] Discharge...
1,224,5350,169684.0,2143-04-30,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2143-4-25**] Discharge...
2,340,59977,127266.0,2143-11-23,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2143-11-17**] ...
3,474,1675,185767.0,2155-07-08,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2155-7-4**] Discharg...
4,1036,55873,137057.0,2193-09-16,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2193-8-29**] ...


In [37]:
notes['DATE'] = pd.to_datetime(notes['CHARTDATE']).dt.date

In [38]:
def clean_text(note: str):
    cleaned = re.sub(r'[^\w]', ' ', note).replace("_", " ")
    removed_spaces = re.sub(' +', ' ', cleaned)
    lower = removed_spaces.lower()
    return lower

In [39]:
notes['CLEAN_TEXT'] = notes['TEXT'].map(clean_text)

In [40]:
notes.head()

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT,DATE,CLEAN_TEXT
0,223,5350,169684.0,2143-04-30,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2143-4-25**] Discharge...,2143-04-30,admission date 2143 4 25 discharge date 2143 4...
1,224,5350,169684.0,2143-04-30,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2143-4-25**] Discharge...,2143-04-30,admission date 2143 4 25 discharge date 2143 4...
2,340,59977,127266.0,2143-11-23,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2143-11-17**] ...,2143-11-23,admission date 2143 11 17 discharge date 2143 ...
3,474,1675,185767.0,2155-07-08,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2155-7-4**] Discharg...,2155-07-08,admission date 2155 7 4 discharge date 2155 7 ...
4,1036,55873,137057.0,2193-09-16,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2193-8-29**] ...,2193-09-16,admission date 2193 8 29 discharge date 2193 9...


### Feature construction

We need to take into account only the events that happened during the observation window. The end of observation window is N days before death for deceased patients and date of last event for alive patients. We can have several sets of events (e.g. labs, diags, meds), so we need to choose the latest date out of those.

In [41]:
OBSERVATION_WINDOW = 2000
PREDICTION_WINDOW = 50

In [42]:
def define_train_period(*feature_sets, 
                        obs_w: int = OBSERVATION_WINDOW, 
                        pred_w: int = PREDICTION_WINDOW) -> (dict, dict):
    """Create SUBJECT_ID -> earliest_date and SUBJECT_ID -> last date dicts."""
    cols = ['SUBJECT_ID', 'DATE']
    all_feats = pd.concat([feats[cols] for feats in feature_sets])
    last_date_base = all_feats.groupby('SUBJECT_ID').DATE.max().to_dict()
    last_date = {subj_id: date
                 for subj_id, date in last_date_base.items()
                 if subj_id not in DECEASED_TO_DATE}
    subtracted_pred_w = {subj_id: date - dt.timedelta(days=pred_w)
                         for subj_id, date in DECEASED_TO_DATE.items()}
    last_date.update(subtracted_pred_w)
    earliest_date = {subj_id: date - dt.timedelta(days=obs_w)
                     for subj_id, date in last_date.items()}
    return earliest_date, last_date

In [43]:
def clean_up_feature_sets(*feature_sets, earliest_date: dict, last_date: dict) -> list:
    """Leave only features from inside the observation window."""
    results = []
    for feats in feature_sets:
        results.append(feats[(feats.DATE < feats.SUBJECT_ID.map(last_date))
                             & (feats.DATE >= feats.SUBJECT_ID.map(earliest_date))])
    return results

In [44]:
use_feature_sets = [diag_preprocessed, lab_preprocessed, meds_preprocessed]

In [45]:
earliest_date, last_date = define_train_period(*use_feature_sets)

In [46]:
diag, lab, med = clean_up_feature_sets(*use_feature_sets, earliest_date=earliest_date, last_date=last_date)

#### Feat calculations

We are going to do a train test split based on patients to validate our model. We will only use those features that appear in the train set. Also, we will only use features that are shared between many patients (we will define "many" manually for each of the feature sets).  

This way we will lose some patients who don't have "popular" features, but that's fine since our goal is to compare similar patients, not to train the best model.

In [47]:
TRAIN_SIZE = 0.8

In [48]:
train_ids, test_ids = train_test_split(list(SAMPLE_IDS), train_size=TRAIN_SIZE, random_state=RANDOM_SEED)

In [49]:
def build_feats(df: pd.DataFrame, agg: list, train_ids: list = None, low_thresh: int = None) -> pd.DataFrame:
    """Build feature aggregations for patient.
    
    Args:
        agg: list of aggregations to use
        train_ids: if not empty, only features that exist in the train set 
            will be used
        low_thresh: if not empty, only features that more than low_thresh
            patients have will be used
    """
    cols_to_use = ['SUBJECT_ID', 'FEATURE_NAME']
    print(f"Total feats: {df.FEATURE_NAME.nunique()}")
    
    if train_ids is not None:
        train_df = df[df.SUBJECT_ID.isin(train_ids)]
        train_feats = set(train_df.FEATURE_NAME)
        df = df[df.FEATURE_NAME.isin(train_feats)]
        print(f"Feats after leaving only train: {len(train_feats)}")
        
    if low_thresh is not None:
        deduplicated = df.drop_duplicates(cols_to_use)
        count = Counter(deduplicated.FEATURE_NAME)
        features_to_leave = set(feat for feat, cnt in count.items() if cnt > low_thresh)
        df = df[df.FEATURE_NAME.isin(features_to_leave)]
        print(f"Feats after removing rare: {len(features_to_leave)}")
    
    grouped = df.groupby(cols_to_use).VALUE.agg(agg)
    return grouped

In [50]:
def pivot_aggregation(df: pd.DataFrame, fill_value = None, use_sparse: bool = True) -> pd.DataFrame:
    """Make sparse pivoted table with SUBJECT_ID as index."""
    pivoted = df.unstack()
    if fill_value is not None:
        pivoted = pivoted.fillna(fill_value)
    
    if use_sparse:
        pivoted = pivoted.astype(pd.SparseDtype("float", fill_value))
    
    pivoted.columns = [f"{col[-1]}_{col[1]}" for col in pivoted.columns]
    return pivoted

In [51]:
diag_built = build_feats(diag, agg=[lambda x: x.sum() > 0], train_ids=train_ids, low_thresh=30)

Total feats: 4472
Feats after leaving only train: 4201
Feats after removing rare: 548


In [52]:
diag_final = pivot_aggregation(diag_built, fill_value=0)

In [53]:
diag_final.head()

,ICD9_00845_ICD9_00845,ICD9_0380_ICD9_0380,ICD9_03811_ICD9_03811,ICD9_03819_ICD9_03819,ICD9_03842_ICD9_03842,ICD9_03849_ICD9_03849,ICD9_0389_ICD9_0389,ICD9_04104_ICD9_04104,ICD9_04111_ICD9_04111,ICD9_04112_ICD9_04112,ICD9_04119_ICD9_04119,ICD9_0413_ICD9_0413,ICD9_0414_ICD9_0414,ICD9_0416_ICD9_0416,ICD9_0417_ICD9_0417,ICD9_04185_ICD9_04185,ICD9_04186_ICD9_04186,ICD9_042_ICD9_042,ICD9_07032_ICD9_07032,ICD9_07054_ICD9_07054,ICD9_07070_ICD9_07070,ICD9_1120_ICD9_1120,ICD9_1122_ICD9_1122,ICD9_135_ICD9_135,ICD9_1550_ICD9_1550,ICD9_1623_ICD9_1623,ICD9_185_ICD9_185,ICD9_1890_ICD9_1890,ICD9_1961_ICD9_1961,ICD9_1970_ICD9_1970,ICD9_1977_ICD9_1977,ICD9_1983_ICD9_1983,ICD9_1985_ICD9_1985,ICD9_19889_ICD9_19889,ICD9_20280_ICD9_20280,ICD9_2113_ICD9_2113,ICD9_2440_ICD9_2440,ICD9_2449_ICD9_2449,ICD9_25000_ICD9_25000,ICD9_25001_ICD9_25001,ICD9_25002_ICD9_25002,ICD9_25012_ICD9_25012,ICD9_25013_ICD9_25013,ICD9_25040_ICD9_25040,ICD9_25050_ICD9_25050,ICD9_25051_ICD9_25051,ICD9_25060_ICD9_25060,ICD9_25061_ICD9_25061,ICD9_25062_ICD9_25062,ICD9_25080_ICD9_25080,ICD9_2536_ICD9_2536,ICD9_2554_ICD9_2554,ICD9_2639_ICD9_2639,ICD9_2662_ICD9_2662,ICD9_2689_ICD9_2689,ICD9_2720_ICD9_2720,ICD9_2724_ICD9_2724,ICD9_2749_ICD9_2749,ICD9_2752_ICD9_2752,ICD9_2753_ICD9_2753,ICD9_27541_ICD9_27541,ICD9_27542_ICD9_27542,ICD9_2760_ICD9_2760,ICD9_2761_ICD9_2761,ICD9_2762_ICD9_2762,ICD9_2763_ICD9_2763,ICD9_2764_ICD9_2764,ICD9_2765_ICD9_2765,ICD9_27650_ICD9_27650,ICD9_27651_ICD9_27651,ICD9_27652_ICD9_27652,ICD9_2766_ICD9_2766,ICD9_27669_ICD9_27669,ICD9_2767_ICD9_2767,ICD9_2768_ICD9_2768,...,ICD9_E8859_ICD9_E8859,ICD9_E8889_ICD9_E8889,ICD9_E912_ICD9_E912,ICD9_E915_ICD9_E915,ICD9_E9289_ICD9_E9289,ICD9_E9308_ICD9_E9308,ICD9_E9320_ICD9_E9320,ICD9_E9331_ICD9_E9331,ICD9_E9342_ICD9_E9342,ICD9_E9352_ICD9_E9352,ICD9_E9478_ICD9_E9478,ICD9_E9500_ICD9_E9500,ICD9_E9503_ICD9_E9503,ICD9_E9504_ICD9_E9504,ICD9_V053_ICD9_V053,ICD9_V08_ICD9_V08,ICD9_V090_ICD9_V090,ICD9_V1005_ICD9_V1005,ICD9_V1011_ICD9_V1011,ICD9_V103_ICD9_V103,ICD9_V1046_ICD9_V1046,ICD9_V1051_ICD9_V1051,ICD9_V1052_ICD9_V1052,ICD9_V1082_ICD9_V1082,ICD9_V1083_ICD9_V1083,ICD9_V1087_ICD9_V1087,ICD9_V1204_ICD9_V1204,ICD9_V1251_ICD9_V1251,ICD9_V1254_ICD9_V1254,ICD9_V1259_ICD9_V1259,ICD9_V1272_ICD9_V1272,ICD9_V1301_ICD9_V1301,ICD9_V153_ICD9_V153,ICD9_V1581_ICD9_V1581,ICD9_V1582_ICD9_V1582,ICD9_V1588_ICD9_V1588,ICD9_V173_ICD9_V173,ICD9_V180_ICD9_V180,ICD9_V290_ICD9_V290,ICD9_V293_ICD9_V293,ICD9_V3000_ICD9_V3000,ICD9_V3001_ICD9_V3001,ICD9_V3101_ICD9_V3101,ICD9_V3401_ICD9_V3401,ICD9_V420_ICD9_V420,ICD9_V422_ICD9_V422,ICD9_V433_ICD9_V433,ICD9_V4364_ICD9_V4364,ICD9_V4365_ICD9_V4365,ICD9_V440_ICD9_V440,ICD9_V441_ICD9_V441,ICD9_V443_ICD9_V443,ICD9_V4501_ICD9_V4501,ICD9_V4502_ICD9_V4502,ICD9_V4511_ICD9_V4511,ICD9_V454_ICD9_V454,ICD9_V4573_ICD9_V4573,ICD9_V4579_ICD9_V4579,ICD9_V4581_ICD9_V4581,ICD9_V4582_ICD9_V4582,ICD9_V4586_ICD9_V4586,ICD9_V4589_ICD9_V4589,ICD9_V462_ICD9_V462,ICD9_V4986_ICD9_V4986,ICD9_V4987_ICD9_V4987,ICD9_V502_ICD9_V502,ICD9_V5861_ICD9_V5861,ICD9_V5865_ICD9_V5865,ICD9_V5866_ICD9_V5866,ICD9_V5867_ICD9_V5867,ICD9_V600_ICD9_V600,ICD9_V641_ICD9_V641,ICD9_V707_ICD9_V707,ICD9_V721_ICD9_V721,ICD9_V8741_ICD9_V8741
SUBJECT_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
22,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

In [54]:
labs_built = build_feats(lab, agg=['mean', 'max', 'min'], train_ids=train_ids, low_thresh=50)

Total feats: 659
Feats after leaving only train: 651
Feats after removing rare: 343


In [55]:
labs_final = pivot_aggregation(labs_built, fill_value=0)

In [56]:
labs_final.head()

,LAB_50800_LAB_50800,LAB_50801_LAB_50801,LAB_50802_LAB_50802,LAB_50803_LAB_50803,LAB_50804_LAB_50804,LAB_50805_LAB_50805,LAB_50806_LAB_50806,LAB_50808_LAB_50808,LAB_50809_LAB_50809,LAB_50810_LAB_50810,LAB_50811_LAB_50811,LAB_50812_LAB_50812,LAB_50813_LAB_50813,LAB_50814_LAB_50814,LAB_50815_LAB_50815,LAB_50816_LAB_50816,LAB_50817_LAB_50817,LAB_50818_LAB_50818,LAB_50819_LAB_50819,LAB_50820_LAB_50820,LAB_50821_LAB_50821,LAB_50822_LAB_50822,LAB_50823_LAB_50823,LAB_50824_LAB_50824,LAB_50825_LAB_50825,LAB_50826_LAB_50826,LAB_50827_LAB_50827,LAB_50828_LAB_50828,LAB_50831_LAB_50831,LAB_50835_LAB_50835,LAB_50836_LAB_50836,LAB_50838_LAB_50838,LAB_50841_LAB_50841,LAB_50842_LAB_50842,LAB_50843_LAB_50843,LAB_50849_LAB_50849,LAB_50852_LAB_50852,LAB_50854_LAB_50854,LAB_50855_LAB_50855,LAB_50856_LAB_50856,LAB_50857_LAB_50857,LAB_50861_LAB_50861,LAB_50862_LAB_50862,LAB_50863_LAB_50863,LAB_50864_LAB_50864,LAB_50866_LAB_50866,LAB_50867_LAB_50867,LAB_50868_LAB_50868,LAB_50871_LAB_50871,LAB_50872_LAB_50872,LAB_50873_LAB_50873,LAB_50874_LAB_50874,LAB_50876_LAB_50876,LAB_50878_LAB_50878,LAB_50879_LAB_50879,LAB_50880_LAB_50880,LAB_50882_LAB_50882,LAB_50883_LAB_50883,LAB_50884_LAB_50884,LAB_50885_LAB_50885,LAB_50887_LAB_50887,LAB_50889_LAB_50889,LAB_50890_LAB_50890,LAB_50891_LAB_50891,LAB_50893_LAB_50893,LAB_50895_LAB_50895,LAB_50896_LAB_50896,LAB_50900_LAB_50900,LAB_50902_LAB_50902,LAB_50903_LAB_50903,LAB_50904_LAB_50904,LAB_50905_LAB_50905,LAB_50906_LAB_50906,LAB_50907_LAB_50907,LAB_50908_LAB_50908,...,LAB_51297_LAB_51297,LAB_51300_LAB_51300,LAB_51301_LAB_51301,LAB_51342_LAB_51342,LAB_51343_LAB_51343,LAB_51344_LAB_51344,LAB_51347_LAB_51347,LAB_51351_LAB_51351,LAB_51352_LAB_51352,LAB_51355_LAB_51355,LAB_51360_LAB_51360,LAB_51362_LAB_51362,LAB_51363_LAB_51363,LAB_51373_LAB_51373,LAB_51375_LAB_51375,LAB_51379_LAB_51379,LAB_51382_LAB_51382,LAB_51383_LAB_51383,LAB_51384_LAB_51384,LAB_51388_LAB_51388,LAB_51398_LAB_51398,LAB_51400_LAB_51400,LAB_51404_LAB_51404,LAB_51411_LAB_51411,LAB_51412_LAB_51412,LAB_51419_LAB_51419,LAB_51424_LAB_51424,LAB_51425_LAB_51425,LAB_51426_LAB_51426,LAB_51427_LAB_51427,LAB_51428_LAB_51428,LAB_51431_LAB_51431,LAB_51434_LAB_51434,LAB_51436_LAB_51436,LAB_51438_LAB_51438,LAB_51439_LAB_51439,LAB_51444_LAB_51444,LAB_51446_LAB_51446,LAB_51447_LAB_51447,LAB_51448_LAB_51448,LAB_51450_LAB_51450,LAB_51453_LAB_51453,LAB_51455_LAB_51455,LAB_51457_LAB_51457,LAB_51458_LAB_51458,LAB_51462_LAB_51462,LAB_51463_LAB_51463,LAB_51464_LAB_51464,LAB_51466_LAB_51466,LAB_51469_LAB_51469,LAB_51474_LAB_51474,LAB_51476_LAB_51476,LAB_51478_LAB_51478,LAB_51479_LAB_51479,LAB_51482_LAB_51482,LAB_51484_LAB_51484,LAB_51486_LAB_51486,LAB_51487_LAB_51487,LAB_51491_LAB_51491,LAB_51492_LAB_51492,LAB_51493_LAB_51493,LAB_51497_LAB_51497,LAB_51498_LAB_51498,LAB_51501_LAB_51501,LAB_51503_LAB_51503,LAB_51505_LAB_51505,LAB_51506_LAB_51506,LAB_51508_LAB_51508,LAB_51512_LAB_51512,LAB_51513_LAB_51513,LAB_51514_LAB_51514,LAB_51516_LAB_51516,LAB_51518_LAB_51518,LAB_51519_LAB_51519,LAB_51523_LAB_51523
SUBJECT_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
11,0.0,0.0,2.0,0.0,25.0,0.0,105.0,1.09,94.0,35.0,11.8,0.0,1.5,0.0,0.0,57.0,0.0,30.0,0.0,7.520,222.0,4.0,0.0,135.0,0.0,627.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14.0,0.0,77.0,0.0,0.0,0.0,14.333333,0.0,0.0,0.0,0.0,0.0,16.0,0.0,0.0,26.777778,0.00,0.00,0.70,0.0,0.0,0.0,0.0,9.337500,0.0,0.0,0.0,101.888889,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,5.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,2.0,0.0,1.015,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
22,0.0,0.0,0.0,0.0,22.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29.0,0.0,7.470,287.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.

In [57]:
meds_built = build_feats(med, agg=['mean', 'count'], train_ids=train_ids, low_thresh=50)

Total feats: 3690
Feats after leaving only train: 3537
Feats after removing rare: 796


In [58]:
meds_final = pivot_aggregation(meds_built, fill_value=0)

In [59]:
meds_final.head()

,MED_000015_MED_000015,MED_000018_MED_000018,MED_000019_MED_000019,MED_000175_MED_000175,MED_000266_MED_000266,MED_000283_MED_000283,MED_000284_MED_000284,MED_000286_MED_000286,MED_000287_MED_000287,MED_000343_MED_000343,MED_000346_MED_000346,MED_000379_MED_000379,MED_000380_MED_000380,MED_000390_MED_000390,MED_000391_MED_000391,MED_000393_MED_000393,MED_000464_MED_000464,MED_000474_MED_000474,MED_000507_MED_000507,MED_000574_MED_000574,MED_000575_MED_000575,MED_000576_MED_000576,MED_000591_MED_000591,MED_000592_MED_000592,MED_000759_MED_000759,MED_001181_MED_001181,MED_001185_MED_001185,MED_001187_MED_001187,MED_001195_MED_001195,MED_001198_MED_001198,MED_001205_MED_001205,MED_001209_MED_001209,MED_001210_MED_001210,MED_001220_MED_001220,MED_001226_MED_001226,MED_001233_MED_001233,MED_001248_MED_001248,MED_001255_MED_001255,MED_001262_MED_001262,MED_001275_MED_001275,MED_001285_MED_001285,MED_001356_MED_001356,MED_001408_MED_001408,MED_001412_MED_001412,MED_001417_MED_001417,MED_001641_MED_001641,MED_001645_MED_001645,MED_001659_MED_001659,MED_001711_MED_001711,MED_001723_MED_001723,MED_001740_MED_001740,MED_001775_MED_001775,MED_001777_MED_001777,MED_001848_MED_001848,MED_001972_MED_001972,MED_001978_MED_001978,MED_001989_MED_001989,MED_002001_MED_002001,MED_002003_MED_002003,MED_002005_MED_002005,MED_002006_MED_002006,MED_002026_MED_002026,MED_002151_MED_002151,MED_002301_MED_002301,MED_002302_MED_002302,MED_002305_MED_002305,MED_002336_MED_002336,MED_002362_MED_002362,MED_002366_MED_002366,MED_002446_MED_002446,MED_002451_MED_002451,MED_002475_MED_002475,MED_002522_MED_002522,MED_002524_MED_002524,MED_002528_MED_002528,...,MED_61005.0_MED_61005.0,MED_61242.0_MED_61242.0,MED_61716.0_MED_61716.0,MED_62823.0_MED_62823.0,MED_6329.0_MED_6329.0,MED_6330.0_MED_6330.0,MED_63337.0_MED_63337.0,MED_6375.0_MED_6375.0,MED_63951.0_MED_63951.0,MED_64182.0_MED_64182.0,MED_64586.0_MED_64586.0,MED_64611.0_MED_64611.0,MED_6508.0_MED_6508.0,MED_6522.0_MED_6522.0,MED_6532.0_MED_6532.0,MED_6541.0_MED_6541.0,MED_6549.0_MED_6549.0,MED_6560.0_MED_6560.0,MED_6561.0_MED_6561.0,MED_6562.0_MED_6562.0,MED_6570.0_MED_6570.0,MED_65758.0_MED_65758.0,MED_6599.0_MED_6599.0,MED_6612.0_MED_6612.0,MED_6645.0_MED_6645.0,MED_66452.0_MED_66452.0,MED_6648.0_MED_6648.0,MED_6649.0_MED_6649.0,MED_6650.0_MED_6650.0,MED_6651.0_MED_6651.0,MED_66525.0_MED_66525.0,MED_66887.0_MED_66887.0,MED_6749.0_MED_6749.0,MED_6751.0_MED_6751.0,MED_6753.0_MED_6753.0,MED_67572.0_MED_67572.0,MED_6776.0_MED_6776.0,MED_6778.0_MED_6778.0,MED_6789.0_MED_6789.0,MED_6817.0_MED_6817.0,MED_7367.0_MED_7367.0,MED_759.0_MED_759.0,MED_7948.0_MED_7948.0,MED_8084.0_MED_8084.0,MED_8148.0_MED_8148.0,MED_8163.0_MED_8163.0,MED_8182.0_MED_8182.0,MED_8205.0_MED_8205.0,MED_8208.0_MED_8208.0,MED_8209.0_MED_8209.0,MED_8210.0_MED_8210.0,MED_8348.0_MED_8348.0,MED_8349.0_MED_8349.0,MED_8398.0_MED_8398.0,MED_8921.0_MED_8921.0,MED_8935.0_MED_8935.0,MED_8937.0_MED_8937.0,MED_9043.0_MED_9043.0,MED_9061.0_MED_9061.0,MED_9066.0_MED_9066.0,MED_9156.0_MED_9156.0,MED_9291.0_MED_9291.0,MED_9298.0_MED_9298.0,MED_9329.0_MED_9329.0,MED_9331.0_MED_9331.0,MED_9344.0_MED_9344.0,MED_9395.0_MED_9395.0,MED_9396.0_MED_9396.0,MED_9509.0_MED_9509.0,MED_9510.0_MED_9510.0,MED_9537.0_MED_9537.0,MED_9588.0_MED_9588.0,MED_9592.0_MED_9592.0,MED_9629.0_MED_9629.0,MED_nan_MED_nan
SUBJECT_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
11,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1000.0,0.0,0.0,240.00,0.0,0.0,0.0,42.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

### Model Training

We will use random forest to automatically incorporate feature interrelations into our model.

In [60]:
from sklearn.ensemble import RandomForestClassifier

In [61]:
feats_to_train_on = [diag_final, meds_final, labs_final]

In [62]:
df_final = pd.concat(feats_to_train_on, axis=1).fillna(0)

In [63]:
target = pd.Series(df_final.index.isin(DECEASED_TO_DATE), index=df_final.index, name='target')

In [64]:
target.value_counts()

False    6456
True     1945
Name: target, dtype: int64

In [65]:
# pd.concat([df_final, target], axis=1).to_csv(os.path.join(PATH_DATASETS, 'diag_med_lab.csv'))

In [66]:
train_loc = df_final.index.isin(train_ids)

In [78]:
def train_and_predict(df: pd.DataFrame, target: pd.Series, train_loc: pd.Series, classifier) -> np.array:
    try:
        classifier.fit(df_final[train_loc], target[train_loc])
        pred = classifier.predict_proba(df_final[~train_loc])[:, 1]
    except AttributeError:
        classifier.fit(df_final[train_loc].sparse.to_dense(), target[train_loc])
        pred = classifier.predict_proba(df_final[~train_loc].sparse.to_dense())[:, 1]
    return pred

In [79]:
cl = RandomForestClassifier(random_state=RANDOM_SEED)

In [80]:
pred = train_and_predict(df_final, target, train_loc, cl)

In [81]:
roc_auc_score(target[~train_loc], pred)

0.8511788337458913

In [82]:
pd.Series(cl.feature_importances_, index=df_final.columns).sort_values(ascending=False).iloc[:10]

LAB_50910_LAB_50910    0.006611
LAB_51006_LAB_51006    0.006443
LAB_51006_LAB_51006    0.006394
LAB_51006_LAB_51006    0.006377
LAB_51277_LAB_51277    0.006079
LAB_50910_LAB_50910    0.006056
LAB_51277_LAB_51277    0.005907
LAB_51249_LAB_51249    0.005898
LAB_51249_LAB_51249    0.005634
LAB_50910_LAB_50910    0.005618
dtype: float64

### Add note TF-IDF

In [83]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [84]:
last_note = clean_up_feature_sets(notes, earliest_date=earliest_date, last_date=last_date)[0]

In [85]:
cols = ['SUBJECT_ID', 'DATE', 'CLEAN_TEXT']
last_note = last_note.sort_values(by=cols, ascending=False).drop_duplicates('SUBJECT_ID')[cols]

In [86]:
last_note = last_note[last_note.SUBJECT_ID.isin(SAMPLE_IDS)]

In [87]:
last_note

,SUBJECT_ID,DATE,CLEAN_TEXT
24200,99283,2125-12-12,2125 12 12 9 30 am chest pa lat clip clip num...
24346,98564,2124-05-02,2124 5 2 8 48 am carot cereb hospital1 clip c...
9148,98402,2111-03-08,title sicu hpi 55f s p fall down the stairs la...
22035,98318,2145-06-08,2145 6 8 4 30 am chest portable ap clip clip ...
20891,98051,2142-04-12,2142 4 12 1 34 pm pelvis ap inlet outlet in o...
...,...,...,...
111,261,2102-01-12,admission date 2101 12 27 discharge date 2102 ...
18346,211,2197-12-27,2197 12 27 2 02 pm chest pa lat clip clip num...
34512,180,2193-12-15,npn agree with coworker initials namepattern4 ...
34682,167,2163-04-12,social work met with mother at the bedside yes...


In [88]:
vectorizer = TfidfVectorizer(max_features=200)
tf_idf = vectorizer.fit_transform(last_note.CLEAN_TEXT)


In [89]:
cols = [f"TFIDF_{feat}" for feat in vectorizer.get_feature_names()]
tf_idf_feats = pd.DataFrame.sparse.from_spmatrix(tf_idf, columns=cols, index=last_note.SUBJECT_ID)

#### Training

In [90]:
# making sure no new rows are added
feats_to_train_on = [diag_final, meds_final, labs_final]
df_final = pd.concat(feats_to_train_on, axis=1).fillna(0)
tf_idf_feats = tf_idf_feats[tf_idf_feats.index.isin(df_final.index)]

In [91]:
feats_to_train_on = [diag_final, meds_final, labs_final, tf_idf_feats]

In [92]:
df_final = pd.concat(feats_to_train_on, axis=1).fillna(0)

In [93]:
target = pd.Series(df_final.index.isin(DECEASED_TO_DATE), index=df_final.index, name='target')

In [94]:
target.value_counts()

False    6456
True     1945
Name: target, dtype: int64

In [95]:
# pd.concat([df_final, target], axis=1).to_csv(os.path.join(PATH_DATASETS, 'diag_med_lab_tf_idf.csv'))

In [96]:
cl = RandomForestClassifier(random_state=RANDOM_SEED)

In [97]:
train_loc = df_final.index.isin(train_ids)

In [98]:
pred = train_and_predict(df_final, target, train_loc, cl)

In [99]:
roc_auc_score(target[~train_loc], pred)

0.8485573996672484

In [100]:
pd.Series(cl.feature_importances_, index=df_final.columns).sort_values(ascending=False).iloc[:10]

LAB_51249_LAB_51249    0.007688
LAB_51006_LAB_51006    0.007251
LAB_51277_LAB_51277    0.006918
LAB_51006_LAB_51006    0.006756
LAB_51006_LAB_51006    0.006081
LAB_51277_LAB_51277    0.005853
LAB_51249_LAB_51249    0.005560
LAB_51277_LAB_51277    0.005441
LAB_50931_LAB_50931    0.005173
LAB_50912_LAB_50912    0.005153
dtype: float64

Better results, mostly from getting patient discharge information from notes.

### Add transformer embeddings

In [101]:
last_note

,SUBJECT_ID,DATE,CLEAN_TEXT
24200,99283,2125-12-12,2125 12 12 9 30 am chest pa lat clip clip num...
24346,98564,2124-05-02,2124 5 2 8 48 am carot cereb hospital1 clip c...
9148,98402,2111-03-08,title sicu hpi 55f s p fall down the stairs la...
22035,98318,2145-06-08,2145 6 8 4 30 am chest portable ap clip clip ...
20891,98051,2142-04-12,2142 4 12 1 34 pm pelvis ap inlet outlet in o...
...,...,...,...
111,261,2102-01-12,admission date 2101 12 27 discharge date 2102 ...
18346,211,2197-12-27,2197 12 27 2 02 pm chest pa lat clip clip num...
34512,180,2193-12-15,npn agree with coworker initials namepattern4 ...
34682,167,2163-04-12,social work met with mother at the bedside yes...


In [102]:
last_note = clean_up_feature_sets(notes, earliest_date=earliest_date, last_date=last_date)[0]

In [103]:
cols = ['SUBJECT_ID', 'DATE', 'TEXT']
last_note = last_note.sort_values(by=cols, ascending=False).drop_duplicates('SUBJECT_ID')[cols]

In [104]:
last_note = last_note[last_note.SUBJECT_ID.isin(SAMPLE_IDS)]

In [105]:
def prepare_text_for_tokenizer(text: str):
    cleaned = ('. ').join(text.splitlines())
    removed_symbols = re.sub('[\[\]\*\_#:?!]+', ' ', cleaned)
    removed_spaces = re.sub(' +', ' ', removed_symbols)
    removed_dots = re.sub('\. \.| \.', '.', removed_spaces)
    removed_duplicated_dots = re.sub('\.+', '.', removed_dots)
    return removed_duplicated_dots

In [106]:
prepare_text_for_tokenizer(last_note['TEXT'].iloc[0])

' 2125-12-12 9 30 AM. CHEST (PA & LAT) Clip Clip Number (Radiology) 33276. Reason To better characterise LLL shadowing. Admitting Diagnosis STROKE;TELEMETRY;TRANSIENT ISCHEMIC ATTACK. Hospital 2 MEDICAL CONDITION. 54 year old woman with R lateral medullary syndrome and LLL shadowing. resolved. REASON FOR THIS EXAMINATION. To better characterise LLL shadowing. FINAL REPORT. HISTORY To assess for left lower lobe shadowing. FINDINGS In comparison with the study of 12-8 , the patient has taken a much. better inspiration. Continued enlargement of the cardiac silhouette, though. no evidence of vascular congestion or pleural effusion. The lungs are. essentially clear. '

In [107]:
last_note['TO_TOK'] = last_note.TEXT.map(prepare_text_for_tokenizer)

In [108]:
last_note = last_note.reset_index(drop=True)

In [109]:
last_note.head()

,SUBJECT_ID,DATE,TEXT,TO_TOK
0,99283,2125-12-12,[**2125-12-12**] 9:30 AM\n CHEST (PA & LAT) ...,2125-12-12 9 30 AM. CHEST (PA & LAT) Clip Cli...
1,98564,2124-05-02,[**2124-5-2**] 8:48 AM\n CAROT/CEREB [**Hospit...,2124-5-2 8 48 AM. CAROT/CEREB Hospital1 Clip ...
2,98402,2111-03-08,[**2111-3-8**] 10:27 AM\n CT HEAD W/O CONTRAST...,2111-3-8 10 27 AM. CT HEAD W/O CONTRAST Clip ...
3,98318,2145-06-08,[**2145-6-8**] 4:30 AM\n CHEST (PORTABLE AP) ...,2145-6-8 4 30 AM. CHEST (PORTABLE AP) Clip Cl...
4,98051,2142-04-12,"[**2142-4-12**] 1:34 PM\n PELVIS (AP, INLET & ...","2142-4-12 1 34 PM. PELVIS (AP, INLET & OUTLET..."


In [110]:
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoConfig

In [111]:
import torch

In [112]:
tokenizer = AutoTokenizer.from_pretrained("deepset/covid_bert_base")

In [113]:
config = AutoConfig.from_pretrained('deepset/covid_bert_base', output_hidden_states=True, output_attentions=True)

In [114]:
model = AutoModelForMaskedLM.from_pretrained("deepset/covid_bert_base", config=config)
model.eval()

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [115]:
def get_vector_for_text(text):
    """This is ugly and slow."""
    encoding = tokenizer(text, 
                         add_special_tokens=True, 
                         truncation=True, 
                         padding="max_length", 
                         return_attention_mask=True, 
                         return_tensors="pt")
    with torch.no_grad():
        outputs = model(**encoding)
        hs = outputs.hidden_states
        token_embeddings = torch.stack(hs, dim=0)
        token_embeddings = torch.squeeze(token_embeddings, dim=1)
        token_embeddings = token_embeddings.permute(1,0,2)
        token_vecs = hs[-2][0]
        text_embedding = torch.mean(token_vecs, dim=0)
        return text_embedding

In [116]:
%%time
sentence = get_vector_for_text(last_note.TO_TOK.iloc[0])

CPU times: user 2.6 s, sys: 170 ms, total: 2.77 s
Wall time: 771 ms


In [117]:
torch.save(sentence, f"../data/embeddings/99283.pt")

In [118]:
for row_num, row in tqdm(last_note.iloc[0:].iterrows()):
    text = row['TO_TOK']
    subj_id = row['SUBJECT_ID']
    embedding = get_vector_for_text(text)
    torch.save(embedding, f"../data/embeddings/{subj_id}.pt")

0it [00:00, ?it/s]

In [119]:
from glob import glob

In [120]:
subj_ids = []
embeds = []
for file in tqdm(glob('../data/embeddings/*')):
    name = file.split('/')[-1]
    subj_id = int(name.split('.')[0])
    embedding = torch.load(file)
    subj_ids.append(subj_id)
    embeds.append(np.array(embedding))

  0%|          | 0/833 [00:00<?, ?it/s]

In [121]:
embed_df = pd.DataFrame(embeds, index=subj_ids)

In [122]:
embed_df.columns = [f"EMBED_{i}" for i in embed_df.columns]

In [123]:
embed_df.head()

,EMBED_0,EMBED_1,EMBED_2,EMBED_3,EMBED_4,EMBED_5,EMBED_6,EMBED_7,EMBED_8,EMBED_9,EMBED_10,EMBED_11,EMBED_12,EMBED_13,EMBED_14,EMBED_15,EMBED_16,EMBED_17,EMBED_18,EMBED_19,EMBED_20,EMBED_21,EMBED_22,EMBED_23,EMBED_24,EMBED_25,EMBED_26,EMBED_27,EMBED_28,EMBED_29,EMBED_30,EMBED_31,EMBED_32,EMBED_33,EMBED_34,EMBED_35,EMBED_36,EMBED_37,EMBED_38,EMBED_39,EMBED_40,EMBED_41,EMBED_42,EMBED_43,EMBED_44,EMBED_45,EMBED_46,EMBED_47,EMBED_48,EMBED_49,EMBED_50,EMBED_51,EMBED_52,EMBED_53,EMBED_54,EMBED_55,EMBED_56,EMBED_57,EMBED_58,EMBED_59,EMBED_60,EMBED_61,EMBED_62,EMBED_63,EMBED_64,EMBED_65,EMBED_66,EMBED_67,EMBED_68,EMBED_69,EMBED_70,EMBED_71,EMBED_72,EMBED_73,EMBED_74,...,EMBED_693,EMBED_694,EMBED_695,EMBED_696,EMBED_697,EMBED_698,EMBED_699,EMBED_700,EMBED_701,EMBED_702,EMBED_703,EMBED_704,EMBED_705,EMBED_706,EMBED_707,EMBED_708,EMBED_709,EMBED_710,EMBED_711,EMBED_712,EMBED_713,EMBED_714,EMBED_715,EMBED_716,EMBED_717,EMBED_718,EMBED_719,EMBED_720,EMBED_721,EMBED_722,EMBED_723,EMBED_724,EMBED_725,EMBED_726,EMBED_727,EMBED_728,EMBED_729,EMBED_730,EMBED_731,EMBED_732,EMBED_733,EMBED_734,EMBED_735,EMBED_736,EMBED_737,EMBED_738,EMBED_739,EMBED_740,EMBED_741,EMBED_742,EMBED_743,EMBED_744,EMBED_745,EMBED_746,EMBED_747,EMBED_748,EMBED_749,EMBED_750,EMBED_751,EMBED_752,EMBED_753,EMBED_754,EMBED_755,EMBED_756,EMBED_757,EMBED_758,EMBED_759,EMBED_760,EMBED_761,EMBED_762,EMBED_763,EMBED_764,EMBED_765,EMBED_766,EMBED_767
5285,-0.798802,0.130756,0.671988,-0.503266,0.321561,0.167486,-0.138109,0.283182,0.142994,-0.086918,0.140263,-0.117940,-0.134491,-0.162193,-0.184334,0.658178,0.637654,-0.340463,-0.512746,0.338170,0.990492,-0.499885,0.328712,0.459036,0.205726,0.165479,-0.164664,-0.676376,-0.046614,0.270833,0.700922,-0.117788,0.293810,-0.341717,0.631551,-0.430869,0.051414,-0.237179,0.380903,0.184331,-0.295622,-0.296313,0.011057,-0.000363,-0.522852,-0.027841,0.378444,-0.294858,-0.329169,-0.059924,-0.602925,-0.385992,0.456208,-0.223731,-0.057904,0.305034,-0.073592,0.166305,-0.382342,-0.245806,0.132887,-0.109450,-0.022951,0.221922,-0.166220,0.096496,0.056278,0.160964,-0.704472,0.355464,0.107311,0.002045,0.788721,0.161504,0.436051,...,-0.121475,-0.185049,0.282081,-0.030035,-0.410694,-0.204711,-0.309379,0.954139,-0.491521,-0.277130,0.209304,-0.029853,-0.255413,1.266172,0.111401,-0.116600,0.157362,-0.695698,-0.132504,0.333457,0.081255,-0.201507,0.728113,-0.056593,-0.100532,-0.303419,0.222861,0.131474,0.369915,-0.008144,0.067196,-0.373120,-0.313959,-0.502534,0.126407,-0.704903,-0.896134,0.150310,0.020640,-0.054072,-0.428490,0.013663,-0.165275,0.029552,-0.315281,0.033213,0.616471,0.111539,-0.015011,0.333240,-0.135848,-0.186889,0.495660,0.194710,0.883840,-0.085824,0.129288,-0.236622,0.412517,-0.154571,-0.272965,-0.093266,-0.716633,-0.475912,-0.737194,-0.524004,-0.109473,0.133466,-0.297956,0.068109,-0.111506,-0.824450,0.068347,0.461592,-0.238279
98051,-0.280688,0.802740,0.548370,-0.282611,-0.028171,0.093071,-0.152611,-0.296869,0.203632,-0.411534,-0.262536,0.046728,-0.139504,-0.321624,0.198940,0.517119,0.097881,-0.324706,-0.091824,0.148899,0.415854,-0.000608,0.032371,0.404749,-0.180330,0.190517,-0.256167,-0.115635,-0.362978,-0.021065,0.484093,-0.030303,0.181302,-0.141996,0.148368,0.034773,-0.032207,-0.273553,0.175443,0.057039,0.067191,-0.204902,-0.011195,-0.037448,-0.826448,-0.058914,0.166248,-0.000723,-0.521033,0.145015,-0.508619,-0.177843,0.515539,-0.122970,-0.487692,0.302740,-0.399523,-0.294083,-0.015211,-0.459082,0.346450,-0.168052,0.055544,0.361819,-0.611940,0.151686,-0.075463,0.279492,-0.188566,0.605380,-0.139250,0.204843,0.707199,0.401628,0.041722,...,-0.489388,0.061825,-0.214151,0.075408,-0.356072,-0.273681,-0.178658,0.096870,-0.390266,0.035978,-0.027068,-0.300614,-0.005389,0.683946,0.088988,-0.256731,0.487871,0.075371,-0.037950,0.301867,-0.305417,0.311921,0.886690,-0.252098,-0.307613,0.092330,0.817284,0.106059,0.724567,0.312704,0.606344,-0.194923,-0.255452,-0.280414,-0.087071,-0.548280,-0.946830,0.028157,0.300649,-0.268860,0.438057,-0.527234,0.052847,0.506

#### Training with embeds

In [124]:
# making sure no new rows are added
feats_to_train_on = [diag_final, meds_final, labs_final]
df_final = pd.concat(feats_to_train_on, axis=1).fillna(0)
embed_df = embed_df[embed_df.index.isin(df_final.index)]

In [125]:
feats_to_train_on = [diag_final, meds_final, labs_final, embed_df]

In [126]:
df_final = pd.concat(feats_to_train_on, axis=1).fillna(0)

In [127]:
target = pd.Series(df_final.index.isin(DECEASED_TO_DATE), index=df_final.index, name='target')

In [128]:
target.value_counts()

False    6456
True     1945
Name: target, dtype: int64

In [129]:
# pd.concat([df_final, target], axis=1).to_csv(os.path.join(PATH_DATASETS, 'diag_med_lab_embed.csv'))

In [130]:
cl = RandomForestClassifier(random_state=RANDOM_SEED)

In [131]:
train_loc = df_final.index.isin(train_ids)

In [132]:
pred = train_and_predict(df_final, target, train_loc, cl)

In [133]:
roc_auc_score(target[~train_loc], pred)

0.8498995657996186

In [134]:
pd.Series(cl.feature_importances_, index=df_final.columns).sort_values(ascending=False).iloc[:10]

LAB_51006_LAB_51006    0.015927
LAB_51006_LAB_51006    0.013820
LAB_51277_LAB_51277    0.008231
LAB_50912_LAB_50912    0.006327
LAB_51277_LAB_51277    0.006261
LAB_50912_LAB_50912    0.005945
LAB_51256_LAB_51256    0.005845
LAB_51249_LAB_51249    0.005781
LAB_51277_LAB_51277    0.005478
LAB_50863_LAB_50863    0.005135
dtype: float64